# 3D-MOT - Paper Stats - EOG Correlation (Time Domain)
More exploratory code and figures, with full breakdown of the data is available in the Exploratory Folder.
This notebook 'hardcode' the code for each figure.
This code isn't meant to be pretty and scalable, but rather easy to manipulate to generate specific figures.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import pickle
import copy
import time
import scipy.stats as stats
from tqdm import tqdm

import mne
from mne import Epochs, pick_channels, find_events
from mne.io import read_raw_bdf, concatenate_raws
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch

import collections
from autoreject import AutoReject

from joblib import Parallel, delayed

import sys

import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline

#mne.viz.set_3d_backend('pyvistaqt')

In [2]:
#==================================================================
# General Variables
#==================================================================
# Triggers
# 1 - New Spheres.
# 2 - Red Spheres.
# 4 - Start Moving.
# 8 - Stop Moving.
# 16 - TBD (~4 of them... and 4 blocks.)
# 32 - TBD (juste juste avant le 1, so I guess fin de trial, I have 88 of them... 4 trials)
trigger = 2

conds = ['1', '2', '3']
sides = ['right', 'left', 'classicmod']
perfs = ['good', 'bad']

# Making Custom Triggers and Event ID
# ------------------------------------
custom_trigger = dict()
custom_trigger['1'] = 100
custom_trigger['2'] = 200
custom_trigger['3'] = 300
custom_trigger['right'] = 10
custom_trigger['left'] = 20
custom_trigger['classicmod'] = 30
custom_trigger['good'] = 1
custom_trigger['bad'] = 2

# Making Event ID Dict to make it easy and readable to access (the desired) epochs.
event_id = dict()
for cond in conds:
    for side in sides:
        for perf in perfs:
            event_id['{}/{}/{}'.format(cond, side, perf)] = custom_trigger[cond] + custom_trigger[side] + custom_trigger[perf]
# ------------------------------------

# Channels & Clusters of Channels
# ------------------------------------
channels = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 
       'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 
       'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 
       'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 
       'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']

clusters_chs = dict()
clusters_chs['Frontal'] = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'Fpz', 
                       'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8',  'FT8']
clusters_chs['Central']  = ['C1', 'C3', 'C5', 'C2', 'C4', 'C6', 'Cz', 'CP5', 'CP3', 'CP1', 
                        'CPz', 'CP6', 'CP4', 'CP2', 'FC5', 'FC3', 'FC1', 'FC6', 'FC4', 'FC2']
clusters_chs['Temporal'] = ['FT7', 'T7', 'TP7', 'FT8', 'T8', 'TP8']
clusters_chs['Parietal'] = ['TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 
                        'POz', 'Pz', 'CPz', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4']
clusters_chs['Occipital'] = ['O1', 'Oz', 'POz','O2']

clusters_names = {'F':'Frontal', 'P':'Parietal', 'C':'Central', 'O':'Occipital', 'T':'Temporal'}
# ------------------------------------

# 3D-MOT Phases Information
phases = ['ID', 'Tracking', 'Recall']
stimx = {'ID':0, 'Tracking':2, 'Recall':11}
time_windows = {'ID':(-.2,2), 'Tracking':(1.5, 11), 'Recall':(10.5, 18)}

# Epoch Information
epoch_length = 20
epoch_tmin = -3
baseline_corr = (-1, -0.2)

# Folders & Paths
# directory_src = '/Users/nick/Documents/PhD/CLCT/NeuroTracker/Paper/src/'
# directory_data = '/Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Data/'
# directory_preproc = '/Users/nick/Documents/PhD/CLCT/NeuroTracker/Paper/data/'
#directory_src = 'D:\\PhD\\NeuroTracker - Paper\\src\\'
#directory_data = 'F:\\CAE - NeuroTracker\\Data\\'
#directory_preproc = 'D:\\PhD\\NeuroTracker - Paper\\data\\'

directory_data = '/Volumes/Nick - Data/PhD - Data/CAE - NeuroTracker - EEG/'
directory_preproc = '/Volumes/Nick - Data/PhD/MOT-EEG/data/'


In [57]:
coeff = np.corrcoef(CDA_all_mean_right, avg_right)
print(f'Correlation for Right Trials: {coeff[0,1]}')

coeff = np.corrcoef(CDA_all_mean_left, avg_left)
print(f'Correlation for Left Trials: {coeff[0,1]}')

Correlation for Right Trials: -0.2430456584924911
Correlation for Left Trials: 0.44551064032933857


# 1. Participant-Level Correlation (Frontal) - EXG NOT in the mean

In [14]:
cda_eog_corr = dict()

total_subjects = 0
total_trials = 0

CDA_lat_all = dict()
#==================================================================
# Main Loop - Loading All Files & Saving Clean Epochs.
#==================================================================
sep = '/'
pbar = tqdm(total=20)
for filepath in glob.iglob(directory_data + '*.bdf'):
    filename = filepath[filepath.rfind(sep)+1:]
    
    # Skipping Exceptions with multiple EEG files. (they will be loaded with Block 1 (B1))
    if filename in ['M8M1_B3.bdf','M8M1_B4.bdf','M8M2_B3.bdf','M8M2_B4.bdf','T9F2_B3-B4.bdf','T9M1_B3-B4.bdf']:
        continue
    
    tic = time.time()
    print('Processing: ' + filename + '...')

    # 1. -- Load File --
    if filename not in ['M8M1_B1-B2.bdf', 'M8M2_B1-B2.bdf', 'T9F2_B1-B2.bdf', 'T9M1_B1-B2.bdf']: # Check for exceptions with multiple EEG files.
        raw = read_raw_bdf(filepath, preload=True)
    else:
        if filename == 'M8M1_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M1_B3.bdf', preload=True)
            raw3 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M1_B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2, raw3])
            filename = 'M8M1.bdf'
        if filename == 'M8M2_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M2_B3.bdf', preload=True)
            raw3 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M2_B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2, raw3])
            filename = 'M8M2.bdf'
        if filename == 'T9F2_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'T9F2_B3-B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2])
            filename = 'T9F2.bdf'
        if filename == 'T9M1_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'T9M1_B3-B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2])
            filename = 'T9M1.bdf'    

    raw = raw.drop_channels(ch_names=['EXG1','EXG2','EXG3', 'EXG8'])
    raw = raw.set_eeg_reference(ref_channels=['EXG4', 'EXG5'])
    raw = raw.set_channel_types({'EXG4': 'misc', 'EXG5': 'misc'})
    raw = raw.set_channel_types({'EXG6': 'eeg', 'EXG7': 'eeg'})
    
    # Sampling Frequency
    sfreq = raw.info['sfreq']

    # Events 
    events = find_events(raw)

    # Load Behav Data.
    df_behav = pd.read_csv(directory_data + filename[:filename.rfind('.')] + '.csv', sep=',')

    # Rework Events based on Behav Data. (to add Side and SetSize information)
    new_events = []
    nb_trials = 0
    total_skipped = 0
    for e in events:
        cur_cond = None
        cur_side = None
        cur_perf = None
        if e[2] == 2: # Find Start of Trial
            cur_cond = str(df_behav['NbTargets'].values[nb_trials]).lower()
            cur_side = str(df_behav['Mode'].values[nb_trials]).lower()
            cur_perf = 'good' if (int(df_behav['NbTargets'].values[nb_trials]) == int(df_behav['Results'].values[nb_trials])) else 'bad'
            if cur_cond is not None and cur_side is not None and cur_perf is not None:
                #temp_trigger = int(custom_trigger[cur_cond]) + int(custom_trigger[cur_side]) + int (custom_trigger[cur_perf])
                temp_event = [e[0], e[1], event_id['{}/{}/{}'.format(cur_cond, cur_side, cur_perf)]]
                new_events.append(temp_event) # Make a new list of events with new custom triggers.
            else:
                total_skipped = total_skipped + 1
                print('Skipping this event {}: No Cond or Side or Perf'.format(e))

            nb_trials = nb_trials + 1

            # Check Synch!
            #if (nb_trials + 1) % 10 == 0:
                #print('Check Synch!')

    # Working with modified events from now on.
    events = new_events
    
    total_trials = total_trials + nb_trials
    print("A total of {} trials were added and {} were skipped.".format(nb_trials, total_skipped))

    # Remove Events ID if not in file (e.g. some participants don't have bad trials with ss=1).
    print('==================================================')
    print('Removing Events ID not in EEG File...')
    cur_event_id = copy.deepcopy(event_id)
    newevents_in_file = set(np.array(events)[:,2])
    keys_to_delete = []
    for e in cur_event_id.keys():
        if event_id[e] not in newevents_in_file:
            print('Event: {} ({}), not present.'.format(e, event_id[e]))
            keys_to_delete.append(e)
    for key in keys_to_delete:
        del cur_event_id[key]
    print('==================================================') 
    
    # Calculate average response time.
    resp_time_in_ticks = []
    for i, e in enumerate(events):
        if e[2] == 8:
            if events[i+1,2] == 32:
                resp_time_in_ticks.append(events[i+1,0] - events[i,0])
            elif events[i+1,2] == 16: #Synch?!
                if events[i+2,2] == 32:
                    resp_time_in_ticks.append(events[i+2,0] - events[i,0])
                else:
                    print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))
            else:
                print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))

    mean_rt = np.mean(resp_time_in_ticks)
    median_rt = np.median(resp_time_in_ticks)
    std_rt = np.std(resp_time_in_ticks)
    print('Response Time ~{:.2f}s (std:{:.2f})'.format(mean_rt/sfreq, std_rt/sfreq))
        
    # Epoching
    epochs = Epochs(raw, events, event_id=cur_event_id, tmin=-2, tmax=15, baseline=(-1, -0.2), preload=True, event_repeated='merge', on_missing='warn')
    
    epochs = epochs.filter(1, 30)
    
    # Get Left and Right Channel in the cluster.
    clusters_chs_right = []
    clusters_chs_left = []
    for ch in clusters_chs['Frontal']:
        if ch[-1] != 'z':
            if int(ch[-1]) % 2 == 0:
                clusters_chs_right.append(ch)
            else:
                clusters_chs_left.append(ch)

    cda_eog_corr[filename] = dict()
    cda_eog_corr[filename]['right'] = dict()
    cda_eog_corr[filename]['right']['right'] = dict()
    cda_eog_corr[filename]['right']['left'] = dict()
    cda_eog_corr[filename]['left'] = dict()
    cda_eog_corr[filename]['left']['right'] = dict()
    cda_eog_corr[filename]['left']['left'] = dict()
    
    for side in ['left', 'right']:
        e_filter = side + '/' + 'good'
        try:
            cur_evoked_1 = epochs[e_filter].copy().pick_channels(clusters_chs_right).crop(tmin=-0.5, tmax=15).average()
            cur_evoked_2 = epochs[e_filter].copy().pick_channels(clusters_chs_left).crop(tmin=-0.5, tmax=15).average()
                
            if side == 'right':
                cur_cda = (np.mean(cur_evoked_2.get_data(), 0) - np.mean(cur_evoked_1.get_data(), 0))
            else:
                cur_cda = (np.mean(cur_evoked_1.get_data(), 0) - np.mean(cur_evoked_2.get_data(), 0))
            
            # ----------- Correlation Stuff -------------
            cur_eog_exg6 = epochs[e_filter].copy().pick_channels(['EXG6']).crop(tmin=-0.5, tmax=15).average().get_data()
            cur_eog_exg7 = epochs[e_filter].copy().pick_channels(['EXG7']).crop(tmin=-0.5, tmax=15).average().get_data()
            cur_eog_both = cur_eog_exg7 - cur_eog_exg6
            
            cda_eog_corr[filename][side]['right']['EXG6'] = np.corrcoef(np.mean(cur_evoked_1.get_data(), 0), cur_eog_exg6)[0,1]
            cda_eog_corr[filename][side]['left']['EXG6'] = np.corrcoef(np.mean(cur_evoked_2.get_data(), 0), cur_eog_exg6)[0,1]
            cda_eog_corr[filename][side]['right']['EXG7'] = np.corrcoef(np.mean(cur_evoked_1.get_data(), 0), cur_eog_exg7)[0,1]
            cda_eog_corr[filename][side]['left']['EXG7'] = np.corrcoef(np.mean(cur_evoked_2.get_data(), 0), cur_eog_exg7)[0,1]
            cda_eog_corr[filename][side]['both'] = np.corrcoef(cur_cda, cur_eog_both)[0,1]
            # ----------- /Correlation Stuff -------------
            
            times = cur_evoked_1.times

        except:
            print(f'Skipping... {filename} - {size} - {side}')

    # Print Correlation Stuff.
    print('========================================')
    print('CORR (FRONTAL) right-right-EXG6 = {}'.format(cda_eog_corr[filename]['right']['right']['EXG6']))
    print('CORR (FRONTAL) right-right-EXG7 = {}'.format(cda_eog_corr[filename]['right']['right']['EXG7']))
    print('CORR (FRONTAL) right-left-EXG6  = {}'.format(cda_eog_corr[filename]['right']['left']['EXG6']))
    print('CORR (FRONTAL) right-left-EXG7  = {}'.format(cda_eog_corr[filename]['right']['left']['EXG7']))
    print('CORR (FRONTAL) right-both       = {}'.format(cda_eog_corr[filename]['right']['both']))
    
    print('CORR (FRONTAL) left-right-EXG6 = {}'.format(cda_eog_corr[filename]['left']['right']['EXG6']))
    print('CORR (FRONTAL) left-right-EXG7 = {}'.format(cda_eog_corr[filename]['left']['right']['EXG7']))
    print('CORR (FRONTAL) left-left-EXG6  = {}'.format(cda_eog_corr[filename]['left']['left']['EXG6']))
    print('CORR (FRONTAL) left-left-EXG7  = {}'.format(cda_eog_corr[filename]['left']['left']['EXG7']))
    print('CORR (FRONTAL) left-both       = {}'.format(cda_eog_corr[filename]['left']['both']))
    print('========================================')
        
    total_subjects = total_subjects + 1
    
    toc = time.time()
    print('Subject {} Done! ({:.2f}s)'.format(filename[:-4], toc-tic))
    
    pbar.update(1)
    
pbar.close() 
print(f'{total_subjects} Participants with a total of {total_trials}.')

data_pickle = dict()
data_pickle['CDA_lat_all'] = CDA_lat_all
data_pickle['cda_eog_corr'] = cda_eog_corr
data_pickle['times'] = times
with open(directory_preproc + 'CDA/Lateralized_raw-EOG-corr.pkl',"wb") as f:
    pickle.dump(data_pickle, f)


 10%|████                                     | 2/20 [06:44<1:00:39, 202.17s/it]

Processing: S21F1.bdf...
Extracting EDF parameters from /Volumes/Nick - Data/PhD - Data/CAE - NeuroTracker - EEG/S21F1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7776255  =      0.000 ...  3797.000 secs...


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
576 events found
Event IDs: [ 1  2  4  8 16 32]
A total of 94 trials were added and 0 were skipped.
Removing Events ID not in EEG File...
Response Time ~nans (std:nan)
Not setting metadata
94 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 94 events and 34817 original time points ...


/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 bad epochs dropped
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6759 samples (3.300 sec)



...

CORR (FRONTAL) right-right-EXG6 = 0.4552071697566139
CORR (FRONTAL) right-right-EXG7 = 0.5525699054176934
CORR (FRONTAL) right-left-EXG6  = 0.13478525790671067
CORR (FRONTAL) right-left-EXG7  = 0.8114835798966965
CORR (FRONTAL) right-both       = 0.9269629959007634
CORR (FRONTAL) left-right-EXG6 = 0.5937454367354463
CORR (FRONTAL) left-right-EXG7 = 0.3798137064366813
CORR (FRONTAL) left-left-EXG6  = 0.3912485946376512
CORR (FRONTAL) left-left-EXG7  = 0.6163236238998367
CORR (FRONTAL) left-both       = -0.8798035117276052
Subject M8M2 Done! (87.72s)
Processing: S20M1.bdf...
Extracting EDF parameters from /Volumes/Nick - Data/PhD - Data/CAE - NeuroTracker - EEG/S20M1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6948863  =      0.000 ...  3393.000 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to det

/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 bad epochs dropped
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6759 samples (3.300 sec)




100%|███████████████████████████████████████████| 20/20 [25:19<00:00, 75.96s/it]

CORR (FRONTAL) right-right-EXG6 = 0.5983842009060134
CORR (FRONTAL) right-right-EXG7 = 0.4912009931557513
CORR (FRONTAL) right-left-EXG6  = 0.34882725695158445
CORR (FRONTAL) right-left-EXG7  = 0.7602928987883258
CORR (FRONTAL) right-both       = 0.8400392406436539
CORR (FRONTAL) left-right-EXG6 = 0.6922039605307022
CORR (FRONTAL) left-right-EXG7 = 0.381232608985039
CORR (FRONTAL) left-left-EXG6  = 0.35214381649354093
CORR (FRONTAL) left-left-EXG7  = 0.7350213223664356
CORR (FRONTAL) left-both       = -0.8633139434509959
Subject S20M1 Done! (78.63s)
20 Participants with a total of 1617.


In [16]:
data_pickle = dict()
data_pickle['CDA_lat_all'] = CDA_lat_all
data_pickle['cda_eog_corr'] = cda_eog_corr
data_pickle['times'] = times
with open('Lateralized_raw-EOG-corr.pkl',"wb") as f:
    pickle.dump(data_pickle, f)

In [18]:
with open('Lateralized_raw-EOG-corr.pkl',"rb") as f:
    data_pickle = pickle.load(f)
    
CDA_lat_all = data_pickle['CDA_lat_all']
cda_eog_corr = data_pickle['cda_eog_corr']
times = data_pickle['times']

In [19]:
time_window = (times >= -0.2) & (times <= 14)

plt.rcParams.update({'font.size':14})
fig = plt.figure(figsize=(12, 3))

cda_stacked = []
for participant in CDA_lat_all.keys():
    cda_stacked.append(CDA_lat_all[participant]['Frontal']['left'])
CDA_all_stacked_left = np.stack(cda_stacked)
for participant in CDA_lat_all.keys():
    cda_stacked.append(CDA_lat_all[participant]['Frontal']['right'])
CDA_all_stacked_right = np.stack(cda_stacked)

#CDA_all_stacked = np.stack(CDA_all)
CDA_all_mean_right = np.mean(CDA_all_stacked_right, 0)
CDA_all_mean_left = np.mean(CDA_all_stacked_left, 0)

plt.plot(times[time_window], CDA_all_mean_right[time_window] * 1e6, label='Right')
plt.plot(times[time_window], CDA_all_mean_left[time_window] * -1e6, label='Left')
plt.rcParams.update({'font.size':10})
plt.legend()
plt.rcParams.update({'font.size':14})
plt.title(f'Lateralized Activity - Frontal')
plt.ylabel('Amplitude (μV)')
plt.xlabel('Time (s)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

fig.savefig('R2a Time - Lateralized Full Sequence - EXG EOG 1.png', dpi=300, format='png')

ValueError: need at least one array to stack

<Figure size 864x216 with 0 Axes>

In [23]:
cda_right = []
cda_left = []

for filename in cda_eog_corr.keys():

    # Print Correlation Stuff.
    print('========================================')
    print('CORR (FRONTAL) right-right-EXG6 = {}'.format(cda_eog_corr[filename]['right']['right']['EXG6']))
    print('CORR (FRONTAL) right-right-EXG7 = {}'.format(cda_eog_corr[filename]['right']['right']['EXG7']))
    print('CORR (FRONTAL) right-left-EXG6  = {}'.format(cda_eog_corr[filename]['right']['left']['EXG6']))
    print('CORR (FRONTAL) right-left-EXG7  = {}'.format(cda_eog_corr[filename]['right']['left']['EXG7']))
    print('CORR (FRONTAL) right-both       = {}'.format(cda_eog_corr[filename]['right']['both']))

    print('CORR (FRONTAL) left-right-EXG6 = {}'.format(cda_eog_corr[filename]['left']['right']['EXG6']))
    print('CORR (FRONTAL) left-right-EXG7 = {}'.format(cda_eog_corr[filename]['left']['right']['EXG7']))
    print('CORR (FRONTAL) left-left-EXG6  = {}'.format(cda_eog_corr[filename]['left']['left']['EXG6']))
    print('CORR (FRONTAL) left-left-EXG7  = {}'.format(cda_eog_corr[filename]['left']['left']['EXG7']))
    print('CORR (FRONTAL) left-both       = {}'.format(cda_eog_corr[filename]['left']['both']))
    print('========================================')
    
    cda_left.append(cda_eog_corr[filename]['left']['both'])
    cda_right.append(cda_eog_corr[filename]['right']['both'])

print('Mean Left: {:.3f}'.format(np.mean(cda_left, 0)))
print('Mean Right: {:.3f}'.format(np.mean(cda_right, 0)))

CORR (FRONTAL) right-right-EXG6 = 0.6925382506356944
CORR (FRONTAL) right-right-EXG7 = 0.5211995442568528
CORR (FRONTAL) right-left-EXG6  = 0.5036954185607071
CORR (FRONTAL) right-left-EXG7  = 0.7161333514606278
CORR (FRONTAL) right-both       = 0.8418682164809698
CORR (FRONTAL) left-right-EXG6 = 0.44284762645342246
CORR (FRONTAL) left-right-EXG7 = 0.464973724357257
CORR (FRONTAL) left-left-EXG6  = 0.17174910961071077
CORR (FRONTAL) left-left-EXG7  = 0.7036204098441581
CORR (FRONTAL) left-both       = -0.8685036326942059
CORR (FRONTAL) right-right-EXG6 = 0.7502147662332999
CORR (FRONTAL) right-right-EXG7 = 0.5946972103162904
CORR (FRONTAL) right-left-EXG6  = 0.5681421406146174
CORR (FRONTAL) right-left-EXG7  = 0.7933373195190095
CORR (FRONTAL) right-both       = 0.7352146059128061
CORR (FRONTAL) left-right-EXG6 = 0.7635781408583678
CORR (FRONTAL) left-right-EXG7 = 0.5405651645367762
CORR (FRONTAL) left-left-EXG6  = 0.6260247718171852
CORR (FRONTAL) left-left-EXG7  = 0.7077776673608926


# Electrode Pairs Correlation


In [3]:
ch_frontal_pairs = ['Fp1-Fp2', 'AF7-AF8', 'AF3-AF4', 'F1-F2', 'F3-F4', 'F5-F6', 'F7-F8', 'FT7-FT8']

total_subjects = 0
total_trials = 0

CDA_lat_all = dict()
CDA_lat_all_without = dict()
CDA_pair_all = dict()
#==================================================================
# Main Loop - Loading All Files & Saving Clean Epochs.
#==================================================================
sep = '/'
pbar = tqdm(total=20)
for filepath in glob.iglob(directory_data + '*.bdf'):
    filename = filepath[filepath.rfind(sep)+1:]
    
    # Skipping Exceptions with multiple EEG files. (they will be loaded with Block 1 (B1))
    if filename in ['M8M1_B3.bdf','M8M1_B4.bdf','M8M2_B3.bdf','M8M2_B4.bdf','T9F2_B3-B4.bdf','T9M1_B3-B4.bdf']:
        continue
    
    tic = time.time()
    print('Processing: ' + filename + '...')

    # 1. -- Load File --
    if filename not in ['M8M1_B1-B2.bdf', 'M8M2_B1-B2.bdf', 'T9F2_B1-B2.bdf', 'T9M1_B1-B2.bdf']: # Check for exceptions with multiple EEG files.
        raw = read_raw_bdf(filepath, preload=True)
    else:
        if filename == 'M8M1_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M1_B3.bdf', preload=True)
            raw3 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M1_B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2, raw3])
            filename = 'M8M1.bdf'
        if filename == 'M8M2_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M2_B3.bdf', preload=True)
            raw3 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'M8M2_B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2, raw3])
            filename = 'M8M2.bdf'
        if filename == 'T9F2_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'T9F2_B3-B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2])
            filename = 'T9F2.bdf'
        if filename == 'T9M1_B1-B2.bdf':
            raw1 = read_raw_bdf(filepath, preload=True)
            raw2 = read_raw_bdf(filepath[:filepath.rfind(sep)+1] + 'T9M1_B3-B4.bdf', preload=True)
            raw = concatenate_raws([raw1, raw2])
            filename = 'T9M1.bdf'    

    raw = raw.drop_channels(ch_names=['EXG1','EXG2','EXG3', 'EXG8'])
    raw = raw.set_eeg_reference(ref_channels=['EXG4', 'EXG5'])
    raw = raw.set_channel_types({'EXG4': 'misc', 'EXG5': 'misc'})
    raw = raw.set_channel_types({'EXG6': 'eeg', 'EXG7': 'eeg'})
    
    # TODO: DOUBLE CHECK THE EXG4, EXG5 QUALITY ON ALL PARTICIPANTS.

    # Set Electrodes Locations
    #montage = mne.channels.make_standard_montage('biosemi64')
    #raw.set_montage(montage)

    # TODO: Manual Intervention or not?
    # Bad channels - manual removal...
    # Subject T23F1.bdf: PO4
    # Subject T18F2.bdf: Iz
    #raw = raw.drop_channels('PO4')
    #raw = raw.drop_channels('Iz')

    # Sampling Frequency
    sfreq = raw.info['sfreq']

    # Events 
    events = find_events(raw)

    # Load Behav Data.
    df_behav = pd.read_csv(directory_data + filename[:filename.rfind('.')] + '.csv', sep=',')

    # Rework Events based on Behav Data. (to add Side and SetSize information)
    new_events = []
    nb_trials = 0
    total_skipped = 0
    for e in events:
        cur_cond = None
        cur_side = None
        cur_perf = None
        if e[2] == 2: # Find Start of Trial
            cur_cond = str(df_behav['NbTargets'].values[nb_trials]).lower()
            cur_side = str(df_behav['Mode'].values[nb_trials]).lower()
            cur_perf = 'good' if (int(df_behav['NbTargets'].values[nb_trials]) == int(df_behav['Results'].values[nb_trials])) else 'bad'
            if cur_cond is not None and cur_side is not None and cur_perf is not None:
                #temp_trigger = int(custom_trigger[cur_cond]) + int(custom_trigger[cur_side]) + int (custom_trigger[cur_perf])
                temp_event = [e[0], e[1], event_id['{}/{}/{}'.format(cur_cond, cur_side, cur_perf)]]
                new_events.append(temp_event) # Make a new list of events with new custom triggers.
            else:
                total_skipped = total_skipped + 1
                print('Skipping this event {}: No Cond or Side or Perf'.format(e))

            nb_trials = nb_trials + 1

            # Check Synch!
            #if (nb_trials + 1) % 10 == 0:
                #print('Check Synch!')

    # Working with modified events from now on.
    events = new_events
    
    total_trials = total_trials + nb_trials
    print("A total of {} trials were added and {} were skipped.".format(nb_trials, total_skipped))

    # Remove Events ID if not in file (e.g. some participants don't have bad trials with ss=1).
    print('==================================================')
    print('Removing Events ID not in EEG File...')
    cur_event_id = copy.deepcopy(event_id)
    newevents_in_file = set(np.array(events)[:,2])
    keys_to_delete = []
    for e in cur_event_id.keys():
        if event_id[e] not in newevents_in_file:
            print('Event: {} ({}), not present.'.format(e, event_id[e]))
            keys_to_delete.append(e)
    for key in keys_to_delete:
        del cur_event_id[key]
    print('==================================================') 
    
    # Calculate average response time.
    resp_time_in_ticks = []
    for i, e in enumerate(events):
        if e[2] == 8:
            if events[i+1,2] == 32:
                resp_time_in_ticks.append(events[i+1,0] - events[i,0])
            elif events[i+1,2] == 16: #Synch?!
                if events[i+2,2] == 32:
                    resp_time_in_ticks.append(events[i+2,0] - events[i,0])
                else:
                    print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))
            else:
                print('[Error] No Response?! Trigger: {}'.format(events[i+1,2]))

    mean_rt = np.mean(resp_time_in_ticks)
    median_rt = np.median(resp_time_in_ticks)
    std_rt = np.std(resp_time_in_ticks)
    print('Response Time ~{:.2f}s (std:{:.2f})'.format(mean_rt/sfreq, std_rt/sfreq))
        
    # Epoching
    # TODO: take values from const.
    epochs = Epochs(raw, events, event_id=cur_event_id, tmin=-2, tmax=15, baseline=(-1, -0.2), preload=True, event_repeated='merge', on_missing='warn')
    
    epochs = epochs.filter(1, 30)
    #epochs = epochs.apply_baseline(baseline=(-0.2, -0.0))
    
    # Cleaning
    #ar = AutoReject(n_jobs=10)
    #epochs, reject_log = ar.fit_transform(epochs, return_log=True) 
    #fig, ax = plt.subplots(figsize=(15,8))
    #reject_log.plot('horizontal', ax=ax)
    
    # Save Clean Epochs for each Participant.
    #data_pickle = dict()
    #data_pickle['events'] = events
    #data_pickle['epochs'] = epochs
    #with open(directory_preproc + f"cleanAR/Subject_{filename[:-4]}_cleanAR.pkl","wb") as f:
    #    pickle.dump(data_pickle, f, protocol=pickle.HIGHEST_PROTOCOL)

    # Get Left and Right Channel in the cluster.
    clusters_chs_right = []
    clusters_chs_left = []
    for ch in clusters_chs['Frontal']:
        if ch[-1] != 'z':
            if int(ch[-1]) % 2 == 0:
                clusters_chs_right.append(ch)
            else:
                clusters_chs_left.append(ch)
      
    CDA_lat_all[filename] = dict()
    CDA_lat_all_without[filename] = dict()
    CDA_pair_all[filename] = dict()
    for pair in ch_frontal_pairs:
        CDA_lat_all[filename][pair] = dict()
        CDA_lat_all_without[filename][pair] = dict()
        CDA_pair_all[filename][pair] = dict()
        
        for side in ['left', 'right']:
            e_filter = side + '/' + 'good'
            try:
                # ------ WITH Pair in the mean -------
                cur_evoked_1 = epochs[e_filter].copy().pick_channels(clusters_chs_right).crop(tmin=-0.5, tmax=15).average()
                cur_evoked_2 = epochs[e_filter].copy().pick_channels(clusters_chs_left).crop(tmin=-0.5, tmax=15).average()
                print('--------- WITH Pair ----------')
                print(cur_evoked_1.ch_names)
                print(cur_evoked_2.ch_names)
                print('--------- /WITH Pair ----------')
                if side == 'right':
                    cur_cda = (np.mean(cur_evoked_2.get_data(), 0) - np.mean(cur_evoked_1.get_data(), 0))
                else:
                    cur_cda = (np.mean(cur_evoked_1.get_data(), 0) - np.mean(cur_evoked_2.get_data(), 0))

                CDA_lat_all[filename][pair][side] = cur_cda
                
                # ------ WITHOUT Pair in the mean -------
                cur_pair_ch_right = pair[pair.rfind('-')+1:]
                cur_pair_ch_left = pair[0:pair.rfind('-')]
                
                # Remove Channels
                clusters_chs_right.remove(cur_pair_ch_right)
                clusters_chs_left.remove(cur_pair_ch_left)
                
                cur_evoked_1 = epochs[e_filter].copy().pick_channels(clusters_chs_right).crop(tmin=-0.5, tmax=15).average()
                cur_evoked_2 = epochs[e_filter].copy().pick_channels(clusters_chs_left).crop(tmin=-0.5, tmax=15).average()
                print('--------- WITHOUT Pair ----------')
                print(cur_evoked_1.ch_names)
                print(cur_evoked_2.ch_names)
                print('--------- /WITHOUT Pair ----------')                
                if side == 'right':
                    cur_cda = (np.mean(cur_evoked_2.get_data(), 0) - np.mean(cur_evoked_1.get_data(), 0))
                else:
                    cur_cda = (np.mean(cur_evoked_1.get_data(), 0) - np.mean(cur_evoked_2.get_data(), 0))

                CDA_lat_all_without[filename][pair][side] = cur_cda
                
                # Re-add Channels
                clusters_chs_right.append(cur_pair_ch_right)
                clusters_chs_left.append(cur_pair_ch_left)


                # ------ PAIR signal -------
                cur_evoked_1 = epochs[e_filter].copy().pick_channels([cur_pair_ch_right]).crop(tmin=-0.5, tmax=15).average()
                cur_evoked_2 = epochs[e_filter].copy().pick_channels([cur_pair_ch_left]).crop(tmin=-0.5, tmax=15).average()
                print('--------- Just Pair ----------')
                print(cur_evoked_1.ch_names)
                print(cur_evoked_2.ch_names)
                print('--------- /Just Pair ----------')
                if side == 'right':
                    cur_cda = (np.mean(cur_evoked_2.get_data(), 0) - np.mean(cur_evoked_1.get_data(), 0))
                else:
                    cur_cda = (np.mean(cur_evoked_1.get_data(), 0) - np.mean(cur_evoked_2.get_data(), 0))

                CDA_pair_all[filename][pair][side] = cur_cda

                times = cur_evoked_1.times

            except:
                print(f'Skipping... {filename} - {side}')

    total_subjects = total_subjects + 1
    
    toc = time.time()
    print('Subject {} Done! ({:.2f}s)'.format(filename[:-4], toc-tic))
    
    pbar.update(1)
    
pbar.close() 
print(f'{total_subjects} Participants with a total of {total_trials}.')

data_pickle = dict()
data_pickle['CDA_lat_all'] = CDA_lat_all
data_pickle['CDA_lat_all_without'] = CDA_lat_all_without
data_pickle['CDA_pair_all'] = CDA_pair_all
data_pickle['times'] = times
with open(directory_preproc + 'CDA/Lateralized_data_raw-Pairs.pkl',"wb") as f:
    pickle.dump(data_pickle, f)

  0%|                                                    | 0/20 [00:00<?, ?it/s]

Processing: S21F1.bdf...
Extracting EDF parameters from /Volumes/Nick - Data/PhD - Data/CAE - NeuroTracker - EEG/S21F1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7776255  =      0.000 ...  3797.000 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
576 events found
Event IDs: [ 1  2  4  8 16 32]
A total of 94 trials were added and 0 were skipped.
Removing Events ID not in EEG File...
Response Time ~nans (std:nan)
Not setting metadata
94 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 94 events and 34817 original time points ...


/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 bad epochs dropped
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6759 samples (3.300 sec)

--------- WITH Pair ----------
['Fp2', 'AF8', 'AF4', 'F2', 'F4', 'F6', 'F8', 'FT8']
['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7']
--------- /WITH Pair ----------
--------- WITHOUT Pair ----------
['AF8', 'AF4', 'F2', 'F4', 'F6', 'F8', 'FT8']
['AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7']
--------- /WITHOUT Pair ----------
--------- Just Pair ----------
['Fp2']
['Fp1']
--------- /Just Pair ----------
--------- WITH Pair ----------
['Fp2', 'AF8', 'AF4

  5%|██▏                                        | 1/20 [02:16<43:16, 136.67s/it]

...

--------- Just Pair ----------
['FT8']
['FT7']
--------- /Just Pair ----------
Subject M8M2 Done! (171.72s)
Processing: S20M1.bdf...
Extracting EDF parameters from /Volumes/Nick - Data/PhD - Data/CAE - NeuroTracker - EEG/S20M1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6948863  =      0.000 ...  3393.000 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 events found
Event IDs: [ 1  2  4  8 16 32]
A total of 84 trials were added and 0 were skipped.
Removing Events ID not in EEG File...
Event: 2/classicmod/bad (232), not present.
Response Time ~nans (std:nan)
Not setting metadata
84 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 84 events and

/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/nick/opt/miniconda3/envs/clct/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 bad epochs dropped
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6759 samples (3.300 sec)

--------- WITH Pair ----------
['Fp2', 'AF8', 'AF4', 'F2', 'F4', 'F6', 'F8', 'FT8']
['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7']
--------- /WITH Pair ----------
--------- WITHOUT Pair ----------
['AF8', 'AF4', 'F2', 'F4', 'F6', 'F8', 'FT8']
['AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7']
--------- /WITHOUT Pair ----------
--------- Just Pair ----------
['Fp2']
['Fp1']
--------- /Just Pair ----------
--------- WITH Pair ----------
['Fp2', 'AF8', 'AF4

100%|██████████████████████████████████████████| 20/20 [35:23<00:00, 106.19s/it]

--------- WITHOUT Pair ----------
['Fp2', 'AF8', 'AF4', 'F2', 'F4', 'F6', 'F8']
['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7']
--------- /WITHOUT Pair ----------
--------- Just Pair ----------
['FT8']
['FT7']
--------- /Just Pair ----------
Subject S20M1 Done! (150.55s)
20 Participants with a total of 1617.


In [21]:
corr_coeff = dict()
for pair in ch_frontal_pairs:
    corr_coeff[pair] = dict()
    corr_coeff[pair]['left'] = []
    corr_coeff[pair]['right'] = []
        
for filename in CDA_pair_all.keys():
    print(f'=============== {filename} ================')
    for pair in ch_frontal_pairs:
        cur_corr_coeff_left = np.corrcoef(CDA_lat_all[filename][pair]['left'], CDA_pair_all[filename][pair]['left'])[0][1]
        cur_corr_coeff_right = np.corrcoef(CDA_lat_all[filename][pair]['right'], CDA_pair_all[filename][pair]['right'])[0][1]
        
        corr_coeff[pair]['left'].append(cur_corr_coeff_left)
        corr_coeff[pair]['right'].append(cur_corr_coeff_right)
            
        print('----- {} ------'.format(pair))
        print(corr_coeff[pair]['left'])
        print(corr_coeff[pair]['right'])
        print('--------------------')

=============== S21F1.bdf ================
----- Fp1-Fp2 ------
[0.37185591292372483]
[0.447127580881297]
--------------------
----- AF7-AF8 ------
[0.8405750510496184]
[0.8323919033720348]
--------------------
----- AF3-AF4 ------
[0.6634948808249476]
[0.5507195642834872]
--------------------
----- F1-F2 ------
[0.6379499768431853]
[0.6595637434494346]
--------------------
----- F3-F4 ------
[0.6759639610774016]
[0.7213678381903699]
--------------------
----- F5-F6 ------
[0.8646289858112348]
[0.8678161601947998]
--------------------
----- F7-F8 ------
[0.9355978444148481]
[0.9072555739385905]
--------------------
----- FT7-FT8 ------
[0.83313864933993]
[0.8020127663000841]
--------------------
=============== T11F1.bdf ================
----- Fp1-Fp2 ------
[0.37185591292372483, 0.7068370014332543]
[0.447127580881297, 0.6807379512351083]
--------------------
----- AF7-AF8 ------
[0.8405750510496184, 0.8170556164605912]
[0.8323919033720348, 0.8678800860170774]
--------------------
----

In [22]:
for pair in ch_frontal_pairs:
    print('{} Left: {}'.format(pair, np.mean(corr_coeff[pair]['left'], 0)))
    print('{} Right: {}'.format(pair, np.mean(corr_coeff[pair]['right'], 0)))

Fp1-Fp2 Left: 0.6967424620854207
Fp1-Fp2 Right: 0.6615235027631664
AF7-AF8 Left: 0.8904483210699506
AF7-AF8 Right: 0.9008808966763755
AF3-AF4 Left: 0.8156098069351685
AF3-AF4 Right: 0.8128321407429265
F1-F2 Left: 0.757526798153419
F1-F2 Right: 0.7673021903260526
F3-F4 Left: 0.8291126885499864
F3-F4 Right: 0.8455268703751617
F5-F6 Left: 0.9079756765961292
F5-F6 Right: 0.9123491928049392
F7-F8 Left: 0.8870429577603941
F7-F8 Right: 0.9095443696385571
FT7-FT8 Left: 0.8485699793891293
FT7-FT8 Right: 0.8510721644400048


In [23]:
corr_coeff = dict()
for pair in ch_frontal_pairs:
    corr_coeff[pair] = dict()
    corr_coeff[pair]['left'] = []
    corr_coeff[pair]['right'] = []
        
for filename in CDA_pair_all.keys():
    print(f'=============== {filename} ================')
    for pair in ch_frontal_pairs:
        cur_corr_coeff_left = np.corrcoef(CDA_lat_all_without[filename][pair]['left'], CDA_pair_all[filename][pair]['left'])[0][1]
        cur_corr_coeff_right = np.corrcoef(CDA_lat_all_without[filename][pair]['right'], CDA_pair_all[filename][pair]['right'])[0][1]
        
        corr_coeff[pair]['left'].append(cur_corr_coeff_left)
        corr_coeff[pair]['right'].append(cur_corr_coeff_right)
            
        print('----- {} ------'.format(pair))
        print(corr_coeff[pair]['left'])
        print(corr_coeff[pair]['right'])
        print('--------------------')

=============== S21F1.bdf ================
----- Fp1-Fp2 ------
[0.13060455521793013]
[0.17337277870693935]
--------------------
----- AF7-AF8 ------
[0.7487911828637771]
[0.7356087742612042]
--------------------
----- AF3-AF4 ------
[0.5241344455230279]
[0.3870753393547987]
--------------------
----- F1-F2 ------
[0.60190266425285]
[0.6218289796078155]
--------------------
----- F3-F4 ------
[0.6066057739669594]
[0.6560627218468309]
--------------------
----- F5-F6 ------
[0.8119065046202238]
[0.8154663256786366]
--------------------
----- F7-F8 ------
[0.9032892398463978]
[0.8654844645835217]
--------------------
----- FT7-FT8 ------
[0.7669033817083737]
[0.7301590104779736]
--------------------
=============== T11F1.bdf ================
----- Fp1-Fp2 ------
[0.13060455521793013, 0.6166457459063959]
[0.17337277870693935, 0.5926759906179827]
--------------------
----- AF7-AF8 ------
[0.7487911828637771, 0.7218847312109883]
[0.7356087742612042, 0.7983804152967865]
--------------------


----- F1-F2 ------
[0.60190266425285, 0.7077592905795276, 0.7489351128597591, 0.947763116439022, 0.8151464243316364, 0.7936279502938122, 0.5676477693188929, 0.7854659361364771, 0.8599225992215678, 0.7923774405037028, 0.6798173760579671, 0.6459304247402187, 0.7908973047183241, 0.5799681237684411, 0.8208014384187278, 0.6153290530869074]
[0.6218289796078155, 0.6821083191085501, 0.7391555980360905, 0.9282490507324963, 0.7848000796073341, 0.7529174762462423, 0.5550867853090602, 0.8341977987161455, 0.8556067090793803, 0.8152819094233592, 0.7513857516980369, 0.7939454412839263, 0.7263987753941621, 0.6187384544561318, 0.8410288089027604, 0.6134338410800468]
--------------------
----- F3-F4 ------
[0.6066057739669594, 0.7242875884598953, 0.9172891337268506, 0.9563822902485138, 0.7174661654686626, 0.8481418796033657, 0.7449489100905884, 0.8298786305027254, 0.8814136619838877, 0.9007355237975367, 0.6993320114752622, 0.8051939971096735, 0.9046648352318388, 0.45857985701449006, 0.9030862181203101, 

In [27]:
for pair in ch_frontal_pairs:
    coeff_left = np.mean(corr_coeff[pair]['left'], 0)
    coeff_right = np.mean(corr_coeff[pair]['right'], 0)
    print('{} Left: {:.3f}'.format(pair, coeff_left))
    print('{} Right: {:.3f}'.format(pair, coeff_right))
    print('{} Mean: {:.3f}'.format(pair, (coeff_left + coeff_right)/2.0))
    print('--------------------')

Fp1-Fp2 Left: 0.596
Fp1-Fp2 Right: 0.565
Fp1-Fp2 Mean: 0.581
--------------------
AF7-AF8 Left: 0.810
AF7-AF8 Right: 0.836
AF7-AF8 Mean: 0.823
--------------------
AF3-AF4 Left: 0.772
AF3-AF4 Right: 0.767
AF3-AF4 Mean: 0.769
--------------------
F1-F2 Left: 0.735
F1-F2 Right: 0.744
F1-F2 Mean: 0.739
--------------------
F3-F4 Left: 0.789
F3-F4 Right: 0.807
F3-F4 Mean: 0.798
--------------------
F5-F6 Left: 0.873
F5-F6 Right: 0.877
F5-F6 Mean: 0.875
--------------------
F7-F8 Left: 0.828
F7-F8 Right: 0.862
F7-F8 Mean: 0.845
--------------------
FT7-FT8 Left: 0.786
FT7-FT8 Right: 0.791
FT7-FT8 Mean: 0.788
--------------------


In [28]:
# EOG Mean
np.mean([0.762, 0.769], 0)

0.7655000000000001